# 01_extract_metadata_test_intent_summary.ipynb
Test intent classification and summarization

In [ ]:
from sentence_transformers import SentenceTransformer, util
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv("config/.env")

intent_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Define test text
text = '''How can I submit a claim for health reimbursement in 2024? What are the required forms?'''

# Intent examples
intent_examples = {
    'claim_process': ["how to submit a claim", "claim reimbursement form"],
    'case_status': ["check case status", "status of my application"],
    'general_info': ["what are the office hours", "how to contact"]
}

query_emb = intent_model.encode(text, convert_to_tensor=True)
best_intent, best_score = None, 0
for intent, examples in intent_examples.items():
    ex_embs = intent_model.encode(examples, convert_to_tensor=True)
    score = util.pytorch_cos_sim(query_emb, ex_embs).max().item()
    if score > best_score:
        best_score = score
        best_intent = intent
print(f"📌 Detected Intent: {best_intent} (Score: {best_score:.2f})")

# Summary generation
summary_prompt = f"Summarize the following in 1-2 lines: {text}"
summary = client.chat.completions.create(
    model=deployment,
    messages=[{"role": "user", "content": summary_prompt}],
    max_tokens=100
)
print("📝 Summary:", summary.choices[0].message.content.strip())